In [1]:
import pandas as pd
import matplotlib as mpl
import seaborn as sns

In [2]:
df = pd.read_csv("./train.csv")

Small dataset. Will definitely be doing CV.  

  Some prelim observations:  
  GarageYrBlt: why float? only year. Check that no halves, then perhaps convert to int.  
  Lots of categorical encoding to do. For the features:  
  * if no ordinality, choose one-hot (such a small dataset, why be concerned about the extra dimensionality in this case); not target encoding, only so I don't have to think about data leakage)
  * if ordinality, choose label

  All NA have become NaN. Don't know whether that will mean different behaviour elsewhere at all  
    
Compute correlations (R-sq) *before* training to inform feature engineering. This also possible to do in CV framework?

In [3]:
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [39]:
df.loc[df["GarageYrBlt"] % 1 != 0, "GarageYrBlt"].notna()

39      False
48      False
78      False
88      False
89      False
        ...  
1349    False
1407    False
1449    False
1450    False
1453    False
Name: GarageYrBlt, Length: 81, dtype: bool

In [37]:
df.loc[df["GarageYrBlt"] % 1 != 0, "GarageYrBlt"].notna().value_counts()

GarageYrBlt
False    81
Name: count, dtype: int64

In [43]:
df.loc[df["GarageYrBlt"] % 1 != 0, ["GarageYrBlt", "GarageArea"]]

,GarageYrBlt,GarageArea
39,NaN,0
48,NaN,0
78,NaN,0
88,NaN,0
89,NaN,0
...,...,...
1349,NaN,0
1407,NaN,0
1449,NaN,0
1450,NaN,0


&#8595; &#8595; scenario where df[df... will not work: only df.loc[df

In [22]:
df.loc[df["PoolQC"].notna(), "PoolQC"]

197     Ex
810     Fa
1170    Gd
1182    Ex
1298    Gd
1386    Fa
1423    Gd
Name: PoolQC, dtype: object